<a href="https://colab.research.google.com/github/nov05/Google-Colaboratory/blob/master/generative_ai_with_langchain/04_langchain_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Notebook modified by nov05 on 2025-06-13  

In [1]:
%%capture
!pip install langchain langchain_community langchain_openai langchain_google_genai \
    langchain_chroma langchain_core langchain_text_splitters
## Executing time 56s
## ⚠️ Restart session after installation

*Successfully installed asgiref-3.8.1 backoff-2.2.1 bcrypt-4.3.0 chromadb-1.0.12 coloredlogs-15.0.1 dataclasses-json-0.6.7 durationpy-0.10 fastapi-0.115.9 filetype-1.2.0 google-ai-generativelanguage-0.6.18 httptools-0.6.4 httpx-sse-0.4.0 humanfriendly-10.0 kubernetes-33.1.0 langchain_chroma-0.2.4 langchain_community-0.3.25 langchain_core-0.3.65 langchain_google_genai-2.1.5 langchain_openai-0.3.22 langsmith-0.3.45 marshmallow-3.26.1 mmh3-5.1.0 mypy-extensions-1.1.0 onnxruntime-1.22.0 opentelemetry-api-1.34.1 opentelemetry-exporter-otlp-proto-common-1.34.1 opentelemetry-exporter-otlp-proto-grpc-1.34.1 opentelemetry-instrumentation-0.55b1 opentelemetry-instrumentation-asgi-0.55b1 opentelemetry-instrumentation-fastapi-0.55b1 opentelemetry-proto-1.34.1 opentelemetry-sdk-1.34.1 opentelemetry-semantic-conventions-0.55b1 opentelemetry-util-http-0.55b1 overrides-7.7.0 posthog-4.8.0 pydantic-settings-2.9.1 pypika-0.48.9 python-dotenv-1.1.0 starlette-0.45.3 typing-inspect-0.9.0 uvloop-0.21.0 watchfiles-1.0.5*  

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
chapter_folder_path = "/content/drive/MyDrive/Colab Notebooks/20250602_langchai with generative ai/chapter7"
pickle_path = "/content/drive/MyDrive/Colab Notebooks/20250602_langchai with generative ai/chapter7/documents.pkl"
## Check the documents pickled file
!ls -1 "{chapter_folder_path}" | grep "pkl"
## Count the number of files and directories in the folder
!ls -1 "{chapter_folder_path}" | wc -l

documents.pkl
9


In [ ]:
# # setting the environment variables
# import sys
# import os
# sys.path.insert(0, os.path.abspath('..'))
# from config import set_environment
# set_environment()

In [1]:
import os
os.environ["USER_AGENT"] = "nov05-langchain-docusaurus/0.1"

# 🟢 **Build a RAG on a documentation website**  

* `Docusaurus` is a popular open-source static site generator optimized for building documentation websites (used by `Meta`, `React`, `Redux`, etc.). It typically stores docs as markdown files in a `docs/` directory.  

* **ChatGPT:**  

  The `USER_AGENT` environment variable is used by `langchain_community` (and underlying HTTP clients like `requests`) to **identify your application** when making web requests—just like how a browser says "I'm Chrome" or "I'm Firefox".

  * Recommended Best Practice

    Set a meaningful value that identifies **you or your application**, in a polite and responsible way. This helps server admins understand who’s accessing their site and why—especially important for ethical scraping.

  * Example Values

| Purpose                     | Example `USER_AGENT` Value                                                   |
| --------------------------- | ---------------------------------------------------------------------------- |
| Personal project / testing  | `"MyName-langchain-docusaurus/0.1"`                                          |
| Organization usage          | `"AcmeCorpLangchainBot/1.0 (contact: dev@acmecorp.com)"`                     |
| Research purpose            | `"ResearchBot-LangChain/1.0 (university.edu; contact: name@university.edu)"` |
| Anonymous (not recommended) | `"langchain/0.1"`                                                            |

* **ChatGPT:**  

  `nest_asyncio.apply()` makes it safe to call `DocusaurusLoader.load()` (which may run async code under the hood) in environments where the event loop is already running, avoiding a RuntimeError.

* Check OpenAI billing at https://platform.openai.com/settings/organization/billing/overview. Make sure you have enough credit balance to convert the documents to embeddings.      

  * `text-embedding-3-small`: \$0.02 per 1M tokens   
  * `text-embedding-3-large`: \$0.13 per 1M tokens    
  
  <img src="https://raw.githubusercontent.com/nov05/pictures/refs/heads/master/generative_ai_with_langchain/2025-06-13%2015_27_23-openai%20billing.jpg" width=800>  

In [10]:
## ⚠️ If you have pickled the documents (~30 MB), load from Google Drive file and skip the next cell
## Or download the file at https://drive.google.com/file/d/1YsdZa7NsO-LmwNyTN0VKKDuXYS2yQi6U
## !wget https://drive.google.com/uc?id=1YsdZa7NsO-LmwNyTN0VKKDuXYS2yQi6U -O document.pkl
# import pickle
# with open(pickle_path, "rb") as f:
#     documents = pickle.load(f)

In [2]:
%%time
import nest_asyncio
from langchain_community.document_loaders import DocusaurusLoader
nest_asyncio.apply()
loader = DocusaurusLoader("https://python.langchain.com")
documents = loader.load()
## Fetching pages: 100%|##########| 1558/1558 [04:31<00:00,  5.75it/s]
## CPU times: user 4min 38s, sys: 8.88 s, total: 4min 46s
## Wall time: 9min 6s

Fetching pages: 100%|##########| 1558/1558 [01:53<00:00, 13.78it/s]


CPU times: user 4min 28s, sys: 13.1 s, total: 4min 41s
Wall time: 6min 19s


In [11]:
## Verify loading
documents[0]

Document(metadata={'source': 'https://python.langchain.com/search/', 'loc': 'https://python.langchain.com/search/', 'changefreq': 'weekly', 'priority': '0.5'}, page_content='\n\n\n\n\nSearch the documentation | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchSearch the documentationCommunityTwitterGitHubOrganizationPythonJS/TSMoreHomepageBlogYouTubeCopyright © 2025 LangChain, Inc.\n\n')

In [12]:
## Check document
documents[0].metadata

{'source': 'https://python.langchain.com/search/',
 'loc': 'https://python.langchain.com/search/',
 'changefreq': 'weekly',
 'priority': '0.5'}

In [13]:
## Check document
import re
print(re.sub(r'\n+', '\n', documents[0].page_content.strip()))

Search the documentation | 🦜️🔗 LangChain
Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchSearch the documentationCommunityTwitterGitHubOrganizationPythonJS/TSMoreHomepageBlogYouTubeCopyright © 2025 LangChain, Inc.


In [11]:
# %%time
# ## Pickle the documents
# import sys, pickle
# print(f"Size of documents: {sys.getsizeof(documents)} bytes")  ## 12728 bytes
# with open(pickle_path, "wb") as f:
#     pickle.dump(documents, f)  ## ~ 30 MB

12728
CPU times: user 145 ms, sys: 54.7 ms, total: 199 ms
Wall time: 380 ms


In [14]:
%%time
from langchain.embeddings import CacheBackedEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from google.colab import userdata
store = LocalFileStore("./cache/")
## Enable the "OPENAI_API_KEY" secret in Colab
underlying_embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=userdata.get('OPENAI_API_KEY'),  ## ✅ mandatory
)
## Avoiding unnecessary costs by caching the embeddings.
embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings,
    store,
    namespace=underlying_embeddings.model,
)
## CPU times: user 165 ms, sys: 2 ms, total: 167 ms
## Wall time: 520 ms

CPU times: user 135 ms, sys: 2.83 ms, total: 138 ms
Wall time: 499 ms


In [15]:
%%time
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400, # 1000
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
chunks = text_splitter.split_documents(documents)
print(f"Number of chunks: {len(chunks)}")
## Number of chunks: 91389
## CPU times: user 5.26 s, sys: 121 ms, total: 5.38 s
## Wall time: 5.49 s

Number of chunks: 91389
CPU times: user 5.01 s, sys: 130 ms, total: 5.14 s
Wall time: 5.14 s


In [16]:
%%time
## This is when the embedding cache folder gets populated.
from langchain_chroma import Chroma
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings
)
## You might encounter the following error.
##     BadRequestError: Error code: 400 - {'error': {'message': 'Requested 427435 tokens, max 300000 tokens per request',
##     'type': 'max_tokens_per_request', 'param': None, 'code': 'max_tokens_per_request'}}
## CPU times: user 15min 13s, sys: 32.2 s, total: 15min 46s
## Wall time: 16min 21s ⚠️

CPU times: user 14min 25s, sys: 34.5 s, total: 15min
Wall time: 14min 31s


In [17]:
## Count the number of files in the embedding cache folder: 37972
!ls -1 "/content/cache" | wc -l
## Get the folder size in MB: 2526
!du -sm "/content/cache"

37972
2526	/content/cache


In [18]:
from langchain_google_genai import GoogleGenerativeAI
from google.colab import userdata
llm = GoogleGenerativeAI(
    # model_name="gemini-pro"
    model="gemini-2.0-flash-001",
    api_key=userdata.get('GOOGLE_API_KEY'),  ## ✅ mandatory
)

In [23]:
%%time
# from langchain import hub
from langsmith import Client
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = vector_store.as_retriever()
## Check the prompt at https://smith.langchain.com/hub/rlm/rag-prompt
## Either method works.
# prompt = hub.pull(
#     "rlm/rag-prompt",
#     api_key=userdata.get("LANGSMITH_API_KEY")  ## optional, to suppress warnings
# )
client = Client(api_key=userdata.get("LANGSMITH_API_KEY"))
prompt = client.pull_prompt(
    "rlm/rag-prompt",
    include_model=True
)
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)
result = rag_chain.invoke("What is Task Decomposition?")
## CPU times: user 196 ms, sys: 25.8 ms, total: 222 ms
## Wall time: 1.66 s

CPU times: user 111 ms, sys: 10.5 ms, total: 122 ms
Wall time: 1.98 s


In [24]:
import textwrap
print(textwrap.fill(result, width=72))

Task Decomposition is a technique used to break down complex tasks into
smaller, more manageable steps. It often involves prompting models to
"think step by step," allowing for clearer reasoning and better
performance on intricate problems. This can be achieved through methods
like Chain of Thought prompting.
